In [6]:
# IMPORT LIBRARIES

import numpy as np 
import sympy as sp
import matplotlib.pyplot as plt

In [7]:
# DATA 

MM_238 = 238.029 # Uranium-238 molar mass [g/mol]
MM_235 = 235.044 # Uranium-235 molar mass [g/mol]
MM_H2O = 18.015 # Water molar mass [g/mol]
rho_H2O = 1 # Water density [g/cm3]
NAV = 6.022 * 10 ** 23 # Avogadro number [atoms/mol]
rho_235 = 19.1 # Uranium 235 density [g/cm3]
rho_238 = 19.1 # Uranium 238 density [g/cm3]
x_blanket = 0.01 # Ratio NUNat/NUH2O within the blanket
T = 60 # Operating core temperature [°C]
T_0 = 20 # Operating blanket / reflector temperature [°C]
D_Re = 0.16 # Reflector diffusion coefficient [cm]
nu = 2.44 # Neutrons per fission [-]
D_Core = 0.16 # Core diffusion coefficient [cm]
D_Blanket = 0.16 # Blanket diffusion coefficient [cm]
b = 5 # Blanket width [cm]

# Assumption: neglect fast fission contributions in the blanket (one speed approximation)
# For the core composition use the results found in Assignment 1

In [8]:

# FUNCTION AND SYMBOLIC VARIABLES DEFINITION

x = sp.symbols('x')
B = sp.symbols('B', nonnegative = True) 
L_c = sp.symbols('L_c', nonnegative = True)
L_b = sp.symbols('L_b', nonnegative = True)
L_r = sp.symbols('L_r', nonnegative = True)
D = sp.symbols('D_c', nonnegative = True) # Core diffusion coefficient = Reflector diffusion coefficient
D_b = sp.symbols('D_b', nonnegative = True) # Blanket diffusion coefficient
H = sp.symbols('H', nonnegative = True) # Core width
H_b = sp.symbols('H_b', nonnegative = True) # Blanket width
F_c = sp.Function('phi_c', real = True)(x)
F_b = sp.Function('phi_b', real = True)(x)
F_r = sp.Function('phi_r', real = True)(x)

In [9]:
# ODE + BC
# ODE + BC

ODE_c = sp.Eq(sp.diff(sp.diff(F_c, x)) + B** 2 * F_c, 0)
print('ODE FOR THE CORE:')
display(ODE_c)

ODE_b = sp.Eq(sp.diff(sp.diff(F_b, x)) - (1/L_b**2) * F_b, 0)
print('ODE FOR THE BLANKET:')
display(ODE_b)

ODE_r = sp.Eq(sp.diff(sp.diff(F_r, x)) - (1/L_r**2) * F_r, 0)
print('ODE FOR THE REFLECTOR:')
display(ODE_r)

flux_c = sp.dsolve(ODE_c)
print('GENERAL SOLUTION FOR THE FLUX IN THE CORE:')
display(flux_c)
sC1 = sp.symbols('C1')   
sC2 = sp.symbols('C2')

flux_b = sp.dsolve(ODE_b)
sC3 = sp.symbols('C3')
sC4 = sp.symbols('C4')
flux_b = flux_b.subs({sC1: sC3, sC2: sC4})
flux_b = sp.Eq(
    flux_b.lhs,
    sC3 * sp.sinh(x/L_b) + sC4 * sp.cosh(x/L_b)
)
print('GENERAL SOLUTION FOR THE FLUX IN THE BLANKET:')
display(flux_b)

flux_r = sp.dsolve(ODE_r)
sC5 = sp.symbols('C5')
sC6 = sp.symbols('C6')
flux_r = flux_r.subs({sC1: sC5, sC2: sC6})
print('GENERAL SOLUTION FOR THE FLUX IN THE REFLECTOR:')
display(flux_r)

# Symmetry BC
symmetry = sp.Eq(
    sp.diff(flux_c.rhs, x).subs(x, 0),
    0 
)
print('SYMMETRY CONDITION IN THE CORE:')
display(symmetry)

# Bounded BC (da migliorare, non so farla generica...)
flux_r = flux_r.subs(sC6, 0)
print('GENERAL SOLUTION FOR THE FLUX IN THE REFLECTOR AFTER BOUNDED BC:')
display(flux_r)

# Continuity BC at the interface with the blanket
continuity_cb = sp.Eq(
    flux_c.rhs.subs(x, H/2), 
    flux_b.rhs.subs(x, H/2)
)
print('CONTINUITY CONDITION FOR THE FLUX AT THE INTERFACE WITH THE BLANKET:')
display(continuity_cb)

continuity_curr_cb = sp.Eq(
    -D* sp.diff(flux_c.rhs, x).subs(x, H/2),
    -D * sp.diff(flux_b.rhs, x).subs(x, H/2)
    )
print('CONTINUITY CONDITION FOR THE CURRENT AT THE INTERFACE WITH THE BLANKET:')
display(continuity_curr_cb)

# Continuity BC at the interface with the reflector
continuity_br = sp.Eq(
    flux_b.rhs.subs(x, H/2 + H_b), 
    flux_r.rhs.subs(x, H/2 + H_b)
)
print('CONTINUITY CONDITION FOR THE FLUX AT THE INTERFACE WITH THE REFLECTOR:')
display(continuity_br)

continuity_curr_br = sp.Eq(
    -D* sp.diff(flux_b.rhs, x).subs(x, H/2 + H_b), 
    -D * sp.diff(flux_r.rhs, x).subs(x, H/2 + H_b)
    )
print('CONTINUITY CONDITION FOR THE CURRENT AT THE INTERFACE WITH THE BLANKET:')
display(continuity_curr_br)


ODE FOR THE CORE:


Eq(B**2*phi_c(x) + Derivative(phi_c(x), (x, 2)), 0)

ODE FOR THE BLANKET:


Eq(Derivative(phi_b(x), (x, 2)) - phi_b(x)/L_b**2, 0)

ODE FOR THE REFLECTOR:


...

Eq(-D_c*(B*C1*cos(B*H/2) - B*C2*sin(B*H/2)), -D_c*(C3*cosh(H/(2*L_b))/L_b + C4*sinh(H/(2*L_b))/L_b))

CONTINUITY CONDITION FOR THE FLUX AT THE INTERFACE WITH THE REFLECTOR:


Eq(C3*sinh((H/2 + H_b)/L_b) + C4*cosh((H/2 + H_b)/L_b), C5*exp(-(H/2 + H_b)/L_r))

CONTINUITY CONDITION FOR THE CURRENT AT THE INTERFACE WITH THE BLANKET:


Eq(-D_c*(C3*cosh((H/2 + H_b)/L_b)/L_b + C4*sinh((H/2 + H_b)/L_b)/L_b), C5*D_c*exp(-(H/2 + H_b)/L_r)/L_r)